# `gcc` etc

In [1]:
%%service gcc

mkdir -p /home/iot/gcc-demo
cd /home/iot/gcc-demo

cat >main.c <<EOF
#include <stdio.h>
int main() {
   // printf() displays the string inside quotation
   printf("Hello, World!");
   return 0;
}
EOF

gcc main.c
ls -l
./a.out

total 16
-rwxr-xr-x 1 iot iot 9360 May 24 17:06 a.out
-rw-r--r-- 1 iot iot  127 May 24 17:06 main.c
Hello, World!

In [ ]:
!pip install iot-kernel -U

In [2]:
!pip show iot-kernel

Name: iot-kernel
Version: 0.4.14
Summary: Jupyter Kernel for MicroPython
Home-page: https://github.com/iot49/iot-kernel
Author: Bernhard Boser
Author-email: None
License: MIT
Location: /usr/local/lib/python3.8/site-packages
Requires: iot-device, ipynbname, ipykernel, paramiko, argparse
Required-by: 


In [2]:
%%bash

netstat -tulpn | grep LISTEN

tcp        0      0 127.0.0.1:60035         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:61987         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:55461         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:62661         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:63621         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:57159         0.0.0.0:*               LISTEN      44/python
tcp        0      0 127.0.0.1:50695         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:60937         0.0.0.0:*               LISTEN      44/python
tcp        0      0 127.0.0.1:57291         0.0.0.0:*               LISTEN      44/python
tcp        0      0 127.0.0.1:52813         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:64365         0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:52015         0.0.0.0:*               LISTEN

# ssh

https://superuser.com/questions/1634832/how-can-i-execute-a-command-on-a-remote-server-through-scp/

1. Find service name, e.g. `gcc` -> `gcc_3572438_1785491`
2. `ssh -t -p 22222 root@localhost "balena-engine exec -it <service_name> <cmd>`

In [2]:
!echo balena-engine ps | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
CONTAINER ID        IMAGE                                                                   COMMAND                  CREATED             STATUS                 PORTS                    NAMES
97ada7db46dd        353af4faa6a4                                                            "/usr/bin/entry.sh /…"   About an hour ago   Up About an hour       0.0.0.0:8900->8900/tcp   gcc_3572438_1785491
9bb7930fbb0b        8320870c5263                                                            "/usr/bin/entry.sh /…"   About an hour ago   Up About an hour                                iot49_3572436_1785491
88eb2c7ad476        af8b6fa948cd                                                            "/init"                  About an hour ago   Up About an hour       0.0.0.0:8443->8443/tcp   code-server_3572439_1785491
d4a3e9e032fd        registry2.balena-cloud.com/v2/ee8a630b4962f1e2b4ad682dd9468f7a:latest   "/usr/src/app/entry.…

In [14]:
!ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -t -p 22222 root@localhost "balena-engine exec -it gcc_3572438_1785491 printenv"         2>/dev/null | grep BALENA_SERVICE_NAME
!ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -t -p 22222 root@localhost "balena-engine exec -it code-server_3572439_1785491 printenv" 2>/dev/null | grep BALENA_SERVICE_NAME
!ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -t -p 22222 root@localhost "balena-engine exec -it iot49_3572436_1785491 printenv"       2>/dev/null | grep BALENA_SERVICE_NAME

BALENA_SERVICE_NAME=gcc
BALENA_SERVICE_NAME=code-server
BALENA_SERVICE_NAME=iot49


# ssh from Python

* https://stackoverflow.com/questions/3586106/perform-commands-over-ssh-with-python
* [Paramiko](http://www.paramiko.org/)

# Fix Container Names

In [24]:
!ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -t -p 22222 root@localhost "balena-engine exec -it gcc printenv" 2>/dev/null | grep BALENA_SERVICE_NAME

BALENA_SERVICE_NAME=gcc


In [26]:
!echo balena-engine --help | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.

Usage:	balena-engine [OPTIONS] COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/home/root/.balena-engine")
  -c, --context string     Name of the context to use to connect to the
                           daemon (overrides DOCKER_HOST env var and
                           default context set with "docker context use")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/home/root/.balena-engine/ca.pem")
      --tlscert string     Path to TLS cer

In [27]:
!echo balena-engine images | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
REPOSITORY                                                       TAG                      IMAGE ID            CREATED             SIZE
registry2.balena-cloud.com/v2/9cc99a696bc3884d07ac37a68143ab07   delta-0e5e14c5979f7546   8320870c5263        3 hours ago         2.98GB
registry2.balena-cloud.com/v2/dfa8be9261de38cf9fe038faf2491b60   delta-2ff37b0a42888e8b   353af4faa6a4        3 hours ago         1.92GB
registry2.balena-cloud.com/v2/9b3868198b4e7ba06d09ecfb04154dec   delta-e26a1372e8986833   af8b6fa948cd        8 days ago          496MB
registry2.balena-cloud.com/v2/ee8a630b4962f1e2b4ad682dd9468f7a   latest                   6b9eed135003        7 weeks ago         72.3MB
balena/aarch64-supervisor                                        v12.4.6                  6b9eed135003        7 weeks ago         72.3MB
balena-healthcheck-image                                         latest                   a29f45ccde2a        

In [28]:
!echo balena-engine stats --no-stream | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
CONTAINER ID        NAME                CPU %               MEM USAGE / LIMIT     MEM %               NET I/O             BLOCK I/O           PIDS
97ada7db46dd        gcc                 0.00%               67.37MiB / 1.856GiB   3.54%               131kB / 5.9MB       16.6MB / 152kB      3
9bb7930fbb0b        iot49               7.61%               133.9MiB / 1.856GiB   7.04%               0B / 0B             64.1MB / 1.22MB     23
88eb2c7ad476        code-server         0.00%               43.11MiB / 1.856GiB   2.27%               3.64kB / 0B         3.84MB / 6.04MB     25
d4a3e9e032fd        resin_supervisor    0.30%               52.06MiB / 1.856GiB   2.74%               0B / 0B             45.5MB / 1.31MB     11


In [29]:
!echo balena-engine ps | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
CONTAINER ID        IMAGE                                                                   COMMAND                  CREATED             STATUS                 PORTS                    NAMES
97ada7db46dd        353af4faa6a4                                                            "/usr/bin/entry.sh /…"   About an hour ago   Up About an hour       0.0.0.0:8900->8900/tcp   gcc
9bb7930fbb0b        8320870c5263                                                            "/usr/bin/entry.sh /…"   About an hour ago   Up About an hour                                iot49
88eb2c7ad476        af8b6fa948cd                                                            "/init"                  About an hour ago   Up About an hour       0.0.0.0:8443->8443/tcp   code-server
d4a3e9e032fd        registry2.balena-cloud.com/v2/ee8a630b4962f1e2b4ad682dd9468f7a:latest   "/usr/src/app/entry.…"   8 days ago          Up 2 hours (healthy)    

In [32]:
!echo balena-engine port code-server | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
8443/tcp -> 0.0.0.0:8443


In [31]:
!echo balena-engine info | ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 22222 localhost

Pseudo-terminal will not be allocated because stdin is not a terminal.
Client:
 Debug Mode: false

Server:
 Containers: 4
  Running: 4
  Paused: 0
  Stopped: 0
 Images: 5
 Server Version: 19.03.13-dev
 Storage Driver: overlay2
  Backing Filesystem: <unknown>
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: journald
 Cgroup Driver: systemd
 Plugins:
  Volume: local
  Network: bridge host null
  Log: journald json-file local
 Swarm: 
  NodeID: 
  Is Manager: false
  Node Address: 
 Runtimes: bare runc
 Default Runtime: runc
 Init Binary: balena-engine-init
 containerd version: 
 runc version: dc9208a3303feef5b3839f4323d9beb36df0a9dd
 init version: 949e6fa-dirty (expected: fec3683b971d9)
 Kernel Version: 5.4.83-v8
 Operating System: balenaOS 2.73.1+rev1
 OSType: linux
 Architecture: aarch64
 CPUs: 4
 Total Memory: 1.856GiB
 Name: 2d1d51c
 ID: 5PVB:PLEG:77HJ:5ULX:HUHZ:E5HC:27EB:AO7T:CBD2:XMLP:5J4T:CPHI
 Docker Root Dir: /var/lib/docker
 Debug Mode: false
 Registry: http